In [3]:

from sklearn import datasets
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score,classification_report,confusion_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA 
import skfuzzy as fuzz
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import cm   
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import kagglehub
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [4]:

#download dataset
path = kagglehub.dataset_download("maharshipandya/-spotify-tracks-dataset")

#load CSV into a pandas dataframe
dataset = pd.read_csv(f"{path}/dataset.csv")

#see the first rows
#print(dataset.head())
#print(dataset.shape)

# see the features available
print(list(dataset.columns)) 
#print(dataset.iloc[5])


['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']


In [5]:

#remove duplicates, speech-like tracks and rows with missing entries
dataset = dataset.drop_duplicates(subset=["track_name", "artists"], keep="first")
dataset = dataset[dataset['speechiness'] <= 0.66]
dataset = dataset.dropna() 

#number of unique genres after pre-processing
#num_genres = dataset['track_genre'].nunique()


In [6]:

#select relevant features
selected_features = [
    'danceability', 'energy', 'key', 'loudness', 'mode',
    'acousticness', 'instrumentalness', 'valence', 'tempo'
]

#subset the dataset
X = dataset[selected_features].values
y = dataset['track_genre'].values

#create labels for the genres
le = LabelEncoder()
y_id = le.fit_transform(y)
num_genres = len(le.classes_)


In [7]:

#split proportions
test_size = 0.15
val_size = 0.15

#split into train (.7) and temp (.3)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_id, test_size=(test_size + val_size), random_state=42,
)

#split temp into validation (.15) and test (.15)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=(test_size / (test_size + val_size)), random_state=42,
)

print("Train:", len(X_train))
print("Validation:", len(X_val))
print("Test:", len(X_test))



Train: 56337
Validation: 12072
Test: 12073


In [8]:

scaler=StandardScaler()
X_train[:, [2, 3, 8]] = scaler.fit_transform(X_train[:, [2, 3, 8]])
X_val[:, [2, 3, 8]]   = scaler.transform(X_val[:, [2, 3, 8]])
X_test[:, [2, 3, 8]]  = scaler.transform(X_test[:, [2, 3, 8]])

#print(X_train)


In [9]:

#hyperparameters
embedding_dim = 10     #inicialmente 30, reduzimos para 10
num_epochs = 10        #a partir de 7 epochs, loss < 1%
learning_rate = 0.001  #se calhar diminuir para nao perder informação na rede
dropout = 0.2
batch_size = 64

#data conversion to PyTorch tensors
X_train = torch.tensor(X_train, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.long)
X_val = torch.tensor(X_val, dtype = torch.float32)
y_val = torch.tensor(y_val, dtype = torch.long)

#create dataset and loader
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [10]:

#genre encoder (y -> z)
class GenreEncoder(nn.Module):
    def __init__(self, num_genres, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(num_genres, embedding_dim)

    def forward(self, genre_id):
        if genre_id.dim() == 2 and genre_id.size(1) == 1:
            genre_id = genre_id.squeeze(1)
        z = self.embedding(genre_id)  # [batch, embedding_dim]
        return z

#feature encoder (x -> ẑ)
class FeatureEncoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, dropout_prob = 0.3):
        super().__init__()
        self.input = nn.Linear (input_dim, 64)
        self.hidden1 = nn.Linear (64, 64)
        self.hidden2 = nn.Linear (64, 64)
        self.out = nn.Linear (64, embedding_dim)
        self.dropout = nn.Dropout (p = dropout_prob)


    def forward(self, x):
        x = F.relu (self.input(x)); x = self.dropout(x)

        x = F.relu (self.hidden1(x)); x = self.dropout(x)

        x = F.relu (self.hidden2(x)); x = self.dropout(x)

        z_hat = self.out(x)
        return z_hat


In [11]:

genre_embedding = GenreEncoder(num_genres, embedding_dim)

model = FeatureEncoder(
    input_dim = X_train.shape[1], 
    embedding_dim = embedding_dim, dropout_prob = dropout
)

#loss
criterion = nn.CosineSimilarity(dim=1)

#optimizer
optimizer = optim.Adam(
    list(model.parameters()) + list(genre_embedding.parameters()),
    lr = learning_rate
)


AttributeError: module 'torch' has no attribute 'version'

In [ ]:

for epoch in range(num_epochs):
    model.train()
    genre_embedding.train()

    total_loss = 0.0

    for x_feat, y_genre in train_dataloader:
        optimizer.zero_grad()
        
        z = genre_embedding(y_genre)
        
        z_hat = model(x_feat)

        cos_sim = criterion( z_hat, z)
        loss = 1 - cos_sim.mean()

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {avg_loss:.4f}")
    

In [ ]:

#aqui está para visualizar os K generos mais frequentes

model.eval(); genre_embedding.eval()
with torch.no_grad():
    Zhat_val = model(X_val).cpu().numpy()      # [N_val, emb]
    y_val_np = y_val.cpu().numpy()

#top-K genres
K = 10
counts_all = pd.Series(y_id).value_counts()
topK_ids = counts_all.index[:K].to_numpy()
topK_names = le.inverse_transform(topK_ids)

# filter validation to those K
mask = np.isin(y_val_np, topK_ids)
Z = Zhat_val[mask]
y_small = y_val_np[mask]

# subsample for speed if needed
rng = np.random.RandomState(42)
max_points = 4000
if len(Z) > max_points:
    id = rng.choice(len(Z), size=max_points, replace=False)
    Z = Z[id]; y_small = y_small[id]

# normalize (cosine), PCA -> t-SNE
Zn = Z / (np.linalg.norm(Z, axis=1, keepdims=True) + 1e-12)
pca = PCA(n_components=min(50, Z.shape[1]), random_state=42)
Zp = pca.fit_transform(Zn)

perp = min(50, max(5, len(Zp)//200))
tsne = TSNE(n_components=2, perplexity=perp, learning_rate='auto',
            init='pca', metric='cosine', random_state=42)
Z2 = tsne.fit_transform(Zp)

#plot

cmap = cm.get_cmap('tab20', K)

plt.figure(figsize=(9,7))
for i, gid in enumerate(topK_ids):
    m = (y_small == gid)
    plt.scatter(Z2[m,0], Z2[m,1], s=10, color=cmap(i), alpha=0.8, label=topK_names[i])
plt.legend(title="Genres (global Top-K)", bbox_to_anchor=(1.02,1), loc="upper left", fontsize=8)
plt.title(f"t-SNE of $\\hat{{z}}$ (global top {K} genres, metric=cosine, perp={perp})")
plt.xlabel("t-SNE 1"); plt.ylabel("t-SNE 2")
plt.tight_layout(); plt.show()
print("plot ready")

In [ ]:
#fuzzy c-means clustering - sem visualização

# ---- Extract genre embeddings after training ----
genre_embedding.eval()
Zgenres = genre_embedding.embedding.weight.detach().cpu().numpy()   # [num_genres, embedding_dim]

# ---- Normalize (cosine geometry) ----
Zg = Zgenres / (np.linalg.norm(Zgenres, axis=1, keepdims=True) + 1e-12)

# ---- Fuzzy C-Means clustering ----
c = 8       # number of broad clusters
m = 2.0      # fuzziness parameter
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    Zg.T, c=c, m=m, error=1e-6, maxiter=1000, init=None, seed=42
)
print(f"Fuzzy partition coefficient (FPC): {fpc:.3f}")

# ---- Hard cluster assignment + confidence ----
labels_hard = u.argmax(axis=0)    # cluster index for each genre
labels_conf = u.max(axis=0)       # highest membership value (confidence)

# ---- Build table: micro-genre -> cluster info ----
genre_names_all = le.classes_
df_clusters = pd.DataFrame({
    "genre": genre_names_all,
    "cluster": labels_hard,
    "membership": labels_conf
}).sort_values(["cluster", "membership"], ascending=[True, False]).reset_index(drop=True)

print("\nTop rows of fuzzy cluster mapping:")
print(df_clusters.head(50))

# ---- Map every song's genre ID to its cluster ID ----
cluster_of_genre = labels_hard
y_train_cluster = cluster_of_genre[y_train.cpu().numpy()]
y_val_cluster   = cluster_of_genre[y_val.cpu().numpy()]
y_test_cluster  = cluster_of_genre[y_test]